<a href="https://colab.research.google.com/github/kishorsabarishg/Facial-Emotion-Recognition-Model-and-Weights/blob/master/CKPLUSFER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Emotion Recognition on CK+45 Dataset**

# Importing Pre-processing libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Mounting Google Drive to load the dataste

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import the layers of the Network and the model

In [3]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Input, Activation, Dropout
from keras.models import Model

Using TensorFlow backend.


# Extracting the dataste zip file

In [4]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/images.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


# Locating the path of the dataset

In [5]:
base_dir = "/content/"
image_dir = os.path.join(base_dir,"images")

# Initializing Input and Targets

In [6]:
X = []
current_id = 0
label_ids = {}
Y = []

# Pre-processing

In [7]:
for root,dirs,files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ","-").lower()
            #print(label,path)
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            #print(label_ids)
            #X_train.append(path)
            #y_train.append(label)
            pil_image = Image.open(path).convert("L")#convert image into numbers we are using PIL and "L" is ussed for grayscale
            size = (128,128)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            image_array = np.array(final_image, "uint8")
            X.append(image_array)
            Y.append(id_)

# Converting List into an Numpy array

In [8]:
X = np.asarray(X)

In [9]:
l1 = X.shape[1]
l2 = X.shape[2]

In [10]:
X = X.reshape(-1,l1,l2,1)

In [11]:
print(X.shape)

(981, 128, 128, 1)


# Normalizing

In [12]:
X = X/255

In [13]:
#print(X[1])

In [14]:
#print(Y)

# Creating Validation Set

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 42)

# Functional API of KERAS

In [17]:
def Emotion_Recognition(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(128,(5,5), activation = 'relu')(X_input)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
    X = Conv2D(128,(5,5), activation = 'relu')(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
    X = MaxPooling2D(2,2)(X)
    
    X = Conv2D(64,(3,3), activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
    X = Conv2D(64,(3,3), activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
    X = MaxPooling2D(2,2)(X)
       
    X = Conv2D(32,(3,3), activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
    X = Conv2D(32,(3,3), activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
    X = MaxPooling2D(2,2)(X)
    
    X = Flatten()(X)
    
    X = Dense(512, activation = "relu")(X)
    X = Dense(7, activation = "softmax")(X)
    
    model = Model(inputs = X_input, outputs = X, name = "EMOTION_RECOGNIZER")
    
    return model

# Function Call

In [18]:
emo_rec = Emotion_Recognition(X.shape[1:])

# Compiling Model. Optimizer is ADAM, Loss is Sparse Categorical Cross Entropy

In [19]:
r = emo_rec.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

# Training

In [20]:
emo_rec.fit(X_train, Y_train, epochs = 50, batch_size = 32, validation_split = 0.2, shuffle = True)

Train on 627 samples, validate on 157 samples
Epoch 1/50
627/627 [==============================] - 24s 39ms/step - loss: 4.1677 - accuracy: 0.5024 - val_loss: 3.7174 - val_accuracy: 0.0510
Epoch 2/50
627/627 [==============================] - 11s 18ms/step - loss: 0.4994 - accuracy: 0.8278 - val_loss: 1.9514 - val_accuracy: 0.0573
Epoch 3/50
627/627 [==============================] - 11s 18ms/step - loss: 0.1690 - accuracy: 0.9585 - val_loss: 1.9552 - val_accuracy: 0.0573
Epoch 4/50
627/627 [==============================] - 11s 18ms/step - loss: 0.0670 - accuracy: 0.9872 - val_loss: 1.9586 - val_accuracy: 0.0573
Epoch 5/50
627/627 [==============================] - 11s 18ms/step - loss: 0.0302 - accuracy: 0.9936 - val_loss: 1.9589 - val_accuracy: 0.0573
Epoch 6/50
627/627 [==============================] - 11s 18ms/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 1.9594 - val_accuracy: 0.0573
Epoch 7/50
627/627 [==============================] - 11s 18ms/step - loss: 0.0051 - accur

# Predicting on Validation Set

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
pred = np.zeros((7,1))
pred = [[0],[1],[2],[3],[4],[5],[6]]
preds = emo_rec.evaluate(X_test, Y_test)
Y_pred = emo_rec.predict(X_test, batch_size=32, verbose=0)
print(Y_pred.shape)
predictions1 = np.dot(Y_pred,pred)
print(predictions1.shape)
predictions1 = np.array(predictions1, dtype = int)
predict1 = np.ndarray.flatten(predictions1)
print(predict1.shape)
conf_mat = confusion_matrix(Y_test, predict1)
print(predictions1[-1])
print("Confusion Matrix:")
print(conf_mat)
print("Test Loss:", preds[0])
print("Test Accuracy:", preds[1])

197/197 [==============================] - 1s 7ms/step
(197, 7)
(197, 1)
(197,)
[2]
Confusion Matrix:
[[34  0  0  0  0  0  0]
 [ 9 35  0  0  0  0  0]
 [ 0 11 16  0  0  0  0]
 [ 0  0  4  7  0  0  0]
 [ 0  0  0 16  3  0  0]
 [ 0  0  0  0 25 25  0]
 [ 0  0  0  0  0 12  0]]
Test Loss: 0.06884629828240817
Test Accuracy: 0.9796954393386841


# Storing the Model, Weights and Label

In [22]:
json_file = emo_rec.to_json()

In [23]:
with open("Emotion.json", "w") as file:
    file.write(json_file)

In [24]:
emo_rec.save_weights("Emotion_h5_file.h5")

In [25]:
import pickle
with open("lables.pickle", "wb") as f:
    pickle.dump(label_ids, f)